In [4]:
import numpy
import xml.etree.ElementTree as ET

In [6]:
tree = ET.parse('polska')
root = tree.getroot()

## Print nodes

In [73]:
for node in root.findall(".//{http://sndlib.zib.de/network}node"):
    print(node.attrib)
    x = node.find(".//{http://sndlib.zib.de/network}x")
    y = node.find(".//{http://sndlib.zib.de/network}y")
    print(x.text)
    print(y.text)

{'id': 'Gdansk'}
18.6
54.2
{'id': 'Bydgoszcz'}
17.9
53.1
{'id': 'Kolobrzeg'}
16.1
54.2
{'id': 'Katowice'}
18.8
50.3
{'id': 'Krakow'}
19.8
50.0
{'id': 'Bialystok'}
23.1
53.1
{'id': 'Lodz'}
19.4
51.7
{'id': 'Poznan'}
16.8
52.4
{'id': 'Rzeszow'}
21.9
50.0
{'id': 'Szczecin'}
14.5
53.4
{'id': 'Warsaw'}
21.0
52.2
{'id': 'Wroclaw'}
16.9
51.1


## Print links

In [69]:
for link in root.findall(".//{http://sndlib.zib.de/network}link"):
    source = link.find(".//{http://sndlib.zib.de/network}source")
    target = link.find(".//{http://sndlib.zib.de/network}target")
    setupCost = link.find(".//{http://sndlib.zib.de/network}setupCost")
    print(link.attrib)
    print("Source: " + source.text)
    print("Target: " + target.text)
    print("SetupCost: " + setupCost.text)
    
    additionalModules = link.findall(".//{http://sndlib.zib.de/network}addModule")
    for module in additionalModules:
        capacity = module.find(".//{http://sndlib.zib.de/network}capacity")
        cost = module.find(".//{http://sndlib.zib.de/network}cost")
        print("Module")
        print("Capacity: "+ capacity.text)
        print("Cost: " + cost.text)
    print("")

{'id': 'Link_0_10'}
Source: Gdansk
Target: Warsaw
SetupCost: 156.0
Module
Capacity: 155.0
Cost: 156.0
Module
Capacity: 622.0
Cost: 468.0

{'id': 'Link_0_2'}
Source: Gdansk
Target: Kolobrzeg
SetupCost: 272.0
Module
Capacity: 155.0
Cost: 272.0
Module
Capacity: 622.0
Cost: 816.0

{'id': 'Link_1_2'}
Source: Bydgoszcz
Target: Kolobrzeg
SetupCost: 156.0
Module
Capacity: 155.0
Cost: 156.0
Module
Capacity: 622.0
Cost: 468.0

{'id': 'Link_1_7'}
Source: Bydgoszcz
Target: Poznan
SetupCost: 186.0
Module
Capacity: 155.0
Cost: 186.0
Module
Capacity: 622.0
Cost: 558.0

{'id': 'Link_1_10'}
Source: Bydgoszcz
Target: Warsaw
SetupCost: 272.0
Module
Capacity: 155.0
Cost: 272.0
Module
Capacity: 622.0
Cost: 816.0

{'id': 'Link_2_9'}
Source: Kolobrzeg
Target: Szczecin
SetupCost: 237.0
Module
Capacity: 155.0
Cost: 237.0
Module
Capacity: 622.0
Cost: 711.0

{'id': 'Link_3_4'}
Source: Katowice
Target: Krakow
SetupCost: 208.0
Module
Capacity: 155.0
Cost: 208.0
Module
Capacity: 622.0
Cost: 624.0

{'id': 'Link_3_6'

## Print demands

In [79]:
n = 0
for demand in root.findall(".//{http://sndlib.zib.de/network}demand"):
    source = demand.find(".//{http://sndlib.zib.de/network}source")
    target = demand.find(".//{http://sndlib.zib.de/network}target")
    demandValue = demand.find(".//{http://sndlib.zib.de/network}demandValue")
    print(demand.attrib)
    print("Source: " + source.text)
    print("Target: " + target.text)
    print("DemandValue: " + demandValue.text)
    
    admissiblePaths = demand.findall(".//{http://sndlib.zib.de/network}admissiblePath")
    for path in admissiblePaths:
        print(path.attrib)
        n = n+1
        path_links = path.findall(".//{http://sndlib.zib.de/network}linkId")
        for link_id in path_links:
            print(link_id.text)
    print("")
    
"All paths: {}".format(n)

{'id': 'Demand_0_1'}
Source: Gdansk
Target: Bydgoszcz
DemandValue: 195.0
{'id': 'P_0'}
Link_0_2
Link_1_2
{'id': 'P_1'}
Link_0_10
Link_1_10
{'id': 'P_2'}
Link_0_2
Link_2_9
Link_7_9
Link_1_7
{'id': 'P_3'}
Link_0_5
Link_5_10
Link_1_10
{'id': 'P_4'}
Link_0_10
Link_4_10
Link_3_4
Link_3_11
Link_7_11
Link_1_7
{'id': 'P_5'}
Link_0_2
Link_2_9
Link_7_9
Link_7_11
Link_3_11
Link_3_4
Link_4_10
Link_1_10
{'id': 'P_6'}
Link_0_5
Link_5_8
Link_4_8
Link_4_10
Link_1_10

{'id': 'Demand_0_2'}
Source: Gdansk
Target: Kolobrzeg
DemandValue: 158.0
{'id': 'P_0'}
Link_0_2
{'id': 'P_1'}
Link_0_10
Link_1_10
Link_1_2
{'id': 'P_2'}
Link_0_5
Link_5_10
Link_1_10
Link_1_2
{'id': 'P_3'}
Link_0_10
Link_4_10
Link_3_4
Link_3_11
Link_7_11
Link_1_7
Link_1_2
{'id': 'P_4'}
Link_0_10
Link_1_10
Link_1_7
Link_7_9
Link_2_9
{'id': 'P_5'}
Link_0_5
Link_5_8
Link_4_8
Link_4_10
Link_1_10
Link_1_2
{'id': 'P_6'}
Link_0_5
Link_5_10
Link_4_10
Link_3_4
Link_3_11
Link_7_11
Link_1_7
Link_1_2

{'id': 'Demand_0_3'}
Source: Gdansk
Target: Katowi

{'id': 'Demand_5_8'}
Source: Bialystok
Target: Rzeszow
DemandValue: 140.0
{'id': 'P_0'}
Link_5_8
{'id': 'P_1'}
Link_5_10
Link_4_10
Link_4_8
{'id': 'P_2'}
Link_0_5
Link_0_10
Link_4_10
Link_4_8
{'id': 'P_3'}
Link_5_10
Link_6_10
Link_3_6
Link_3_4
Link_4_8
{'id': 'P_4'}
Link_0_5
Link_0_2
Link_1_2
Link_1_10
Link_4_10
Link_4_8
{'id': 'P_5'}
Link_0_5
Link_0_10
Link_6_10
Link_3_6
Link_3_4
Link_4_8
{'id': 'P_6'}
Link_5_10
Link_1_10
Link_1_7
Link_7_11
Link_3_11
Link_3_4
Link_4_8

{'id': 'Demand_5_9'}
Source: Bialystok
Target: Szczecin
DemandValue: 198.0
{'id': 'P_0'}
Link_0_5
Link_0_2
Link_2_9
{'id': 'P_1'}
Link_5_10
Link_0_10
Link_0_2
Link_2_9
{'id': 'P_2'}
Link_0_5
Link_0_10
Link_1_10
Link_1_2
Link_2_9
{'id': 'P_3'}
Link_0_5
Link_0_2
Link_1_2
Link_1_7
Link_7_9
{'id': 'P_4'}
Link_5_8
Link_4_8
Link_4_10
Link_0_10
Link_0_2
Link_2_9
{'id': 'P_5'}
Link_5_10
Link_1_10
Link_1_2
Link_2_9
{'id': 'P_6'}
Link_5_10
Link_0_10
Link_0_2
Link_1_2
Link_1_7
Link_7_9

{'id': 'Demand_5_10'}
Source: Bialystok
Targ

'All paths: 462'